<a href="https://colab.research.google.com/github/akib1162100/AMT_R-D/blob/r%26d/audioToMidi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#load the audio file
audio_file = "/content/Charlie Puth - One Call Away (Acapella - Vocals Only).mp3"


In [28]:
#Note: If you want librosa to suggest a bpm for the song then write 'None' or '0'
estimated_tempo = None

# Audio to Midi

In [70]:
#Import the required libraries
!pip install midiutil
!pip install pretty_midi
import librosa
from librosa import display
from scipy import stats as st
import numpy as np
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import mido
import pretty_midi
from mido import MidiFile, MidiTrack, Message, MetaMessage
from midiutil.MidiFile import MIDIFile
import time
import math

In [30]:
#load the audio, and detect the pitch
y, sr = librosa.load(audio_file, sr = None) #The raw audio data signal is stored in y
duration = librosa.get_duration(y = y, sr = sr)
x = 10
frame_size = 2**x
z = 1/2
hop_length = int(z*frame_size)

y_seg_clean = librosa.effects.split(y = y, frame_length = frame_size, hop_length = hop_length) #Split an audio signal into non-silent intervals.
y_clean = librosa.effects.remix(y = y, intervals= y_seg_clean) #Remix an audio signal by re-ordering time intervals.
harmonic, percussive = librosa.effects.hpss(y_clean) #Decompose an audio time series into harmonic and percussive components.
onset_env = librosa.onset.onset_strength(y = harmonic, sr = sr, hop_length=hop_length) #Compute a spectral flux onset strength envelope.

onset_samples = librosa.onset.onset_detect(y=harmonic,
                                           onset_envelope = onset_env,
                                           sr=sr, units='samples',
                                           hop_length=hop_length,
                                           backtrack=True,
                                           pre_max=20,
                                           post_max=20,
                                           pre_avg=80,
                                           post_avg=80,
                                           delta = 0.001,
                                           wait=0)
onset_boundaries = np.concatenate([onset_samples, [len(harmonic)]])
onset_times = librosa.samples_to_time(onset_boundaries, sr=sr)

#librosa decides the tempo if provided value is None
tempo = int(librosa.beat.tempo(onset_envelope = onset_env, sr = sr )) if estimated_tempo == 0 or estimated_tempo == None else int(estimated_tempo)

def estimate_pitch(segment, sr, fmin=50.0, fmax=2000.0):

    r = librosa.autocorrelate(segment)#stft

    i_min = sr/fmax
    i_max = sr/fmin
    r[:int(i_min)] = 0
    r[int(i_max):] = 0
    i = r.argmax()
    f0 = float(sr)/i
    return f0

def generate_sine(f0, sr, n_duration):
    n = np.arange(n_duration)
    return 0.2*np.sin(2*np.pi*f0*n/float(sr))


def estimate_vol(segment, sr, fmin=50.0, fmax=2000.0):
    vol = librosa.feature.rms(y = segment)
    vol_avg = np.mean(vol)
    return vol_avg

def estimate_pitch_and_generate_sine(x, onset_samples, i, sr):
    n0 = onset_samples[i]
    n1 = onset_samples[i+1]
    f0 = estimate_pitch(x[n0:n1], sr)#segment of the frequencies
    vol = estimate_vol(x[n0:n1], sr)
    return generate_sine(f0, sr, n1-n0) , librosa.hz_to_note(f0), vol, f0

y_pure = np.concatenate([
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[0]
    for i in range(len(onset_boundaries)-1)
])
y_notes = [
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[1]
    for i in range(len(onset_boundaries)-1)
]
y_vol = [
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[2]
    for i in range(len(onset_boundaries)-1)
]
freq = [
    estimate_pitch_and_generate_sine(harmonic, onset_boundaries, i, sr=sr)[3]
    for i in range(len(onset_boundaries)-1)
]



degrees = [librosa.note_to_midi(note) if note!='0' else 0 for i,note in enumerate(y_notes)]
beats_per_sec = tempo/60
start_times_beat = [onset*beats_per_sec for onset in onset_times]
duration_in_beat = [start_times_beat[i]-start_times_beat[i-1] for i in range(1,len(start_times_beat))]
start_times_beat.pop()
norm_vol = [100] * (len(degrees))

<ipython-input-30-2f539008e7ec>:29: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = int(librosa.beat.tempo(onset_envelope = onset_env, sr = sr )) if estimated_tempo == 0 or estimated_tempo == None else int(estimated_tempo)


In [31]:
#1. Extract the unique notes in the song and their occurences
notes_by_pitch = {}
for note, duration in zip(degrees,duration_in_beat):
    if note in notes_by_pitch: #finds the unique notes
        notes_by_pitch[note].append(duration)
    else:
        notes_by_pitch[note] = [duration]

unique_notes = list(librosa.midi_to_note(list(notes_by_pitch.keys())))

#identify occurences of each of these notes(counting the number of times it appears in the song)
note_occurences = []
for key in notes_by_pitch:
  occurences = len(notes_by_pitch[key])
  note_occurences.append(occurences)

#sort them according to note_occurences
note_data = list(zip(unique_notes, note_occurences))
note_soup = sorted(note_data, key = lambda x: x[1], reverse = True)

#Find the note that occurs the most, this will later be used to determine the octave of the chords
most_common_note = note_soup[0][0]


In [32]:
#Dictionary of all possible keys that a song can be in
key_scales = {

    'C Major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'], # https://vitapiano.com/the-complete-guide-to-piano-scales/
    'D Major': ['D', 'E', 'F♯', 'G', 'A', 'B', 'C♯'], # https://vitapiano.com/the-complete-guide-to-piano-scales/
    'E Major': ['E', 'F♯', 'G♯', 'A', 'B', 'C♯', 'D♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'F Major': ['F', 'G', 'A', 'Bb', 'C', 'D', 'E'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'G Major': ['G', 'A', 'B', 'C', 'D', 'E', 'F♯'], # https://vitapiano.com/the-complete-guide-to-piano-scales/
    'A Major': ['A', 'B', 'C♯', 'D', 'E', 'F♯', 'G♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'B Major': ['B', 'C♯', 'D♯', 'E', 'F♯', 'G♯', 'A♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/

    'C♯ Major': ['C♯', 'D♯', 'E♯', 'F♯', 'G♯', 'A♯', 'B♯'],# https://www.piano-keyboard-guide.com/key-of-c-sharp.html
    'D♯ Major': ['D♯', 'E♯', 'F♯♯', 'G♯', 'A♯', 'B♯', 'C♯♯'],# https://themusicambition.com/d-sharp-major-scale/
    'F♯ Major': ['F♯', 'G♯', 'A♯', 'B', 'C♯', 'D♯', 'E♯'],# https://www.piano-keyboard-guide.com/f-sharp-major-scale.html
    'G♯ Major': ['G♯', 'A♯', 'B♯', 'C♯', 'D♯', 'E♯', 'F♯♯'],# https://themusicambition.com/g-sharp-major-scale/
    'A♯ Major': ['A♯', 'B♯', 'C♯♯', 'D♯', 'E♯', 'F♯♯', 'G♯♯'],# https://themusicambition.com/a-sharp-major-scale/

    'C Minor': ['C', 'D', 'Eb', 'F', 'G', 'Ab', 'Bb'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'D Minor': ['D', 'E', 'F', 'G', 'A', 'Bb', 'C'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'E Minor': ['E', 'F♯', 'G', 'A', 'B', 'C', 'D'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'F Minor': ['F', 'G', 'Ab', 'Bb', 'C', 'Db', 'Eb'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'G Minor': ['G', 'A', 'Bb', 'C', 'D', 'Eb', 'F'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'A Minor': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],#  https://vitapiano.com/the-complete-guide-to-piano-scales/
    'B Minor': ['B', 'C♯', 'D', 'E', 'F♯', 'G', 'A'],# https://vitapiano.com/the-complete-guide-to-piano-scales/

    'C♯ Minor': ['C♯', 'D♯', 'E', 'F♯', 'G♯', 'A', 'B'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'D♯ Minor': ['D♯', 'E♯', 'F♯', 'G♯', 'A♯', 'B', 'C♯'],# https://www.piano-keyboard-guide.com/key-of-d-sharp-minor.html
    'F♯ Minor': ['F♯', 'G♯', 'A', 'B', 'C♯', 'D', 'E'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'G♯ Minor': ['G♯', 'A♯', 'B', 'C♯', 'D♯', 'E', 'F♯'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'A♯ Minor': ['A♯', 'B♯', 'C♯', 'D♯', 'E♯', 'F♯', 'G♯'],# https://www.piano-keyboard-guide.com/key-of-a-sharp-minor.html

    'Cb Major': ['Cb', 'Db', 'Eb', 'Fb', 'Gb', 'Ab', 'Bb'], #https://www.basicmusictheory.com/c-flat-major-triad-chords
    'Db Major': ['Db', 'Eb', 'F', 'Gb', 'Ab', 'Bb', 'C'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Eb Major': ['Eb', 'F', 'G', 'Ab', 'Bb', 'C', 'D'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Gb Major': ['Gb', 'Ab', 'Bb', 'Cb', 'Db', 'Eb', 'F'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Ab Major': ['Ab', 'Bb', 'C', 'Db', 'Eb', 'F', 'G'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Bb Major': ['Bb', 'C', 'D', 'Eb', 'F', 'G', 'A'],# https://vitapiano.com/the-complete-guide-to-piano-scales/


    'Bb Minor': ['Bb', 'C', 'Db', 'Eb', 'F', 'Gb', 'Ab'],#  https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Eb Minor': ['Eb', 'F', 'Gb', 'Ab', 'Bb', 'Cb', 'Db'],# https://vitapiano.com/the-complete-guide-to-piano-scales/
    'Ab Minor': ['Ab', 'Bb', 'Cb', 'Db', 'Eb', 'Fb', 'Gb'],# https://www.basicmusictheory.com/a-flat-minor-triad-chords

}

# these are the relative keys
#  ['C Major', 'A Minor']
#  ['D Major', 'B Minor']
#  ['E Major', 'C♯ Minor']
#  ['F Major', 'D Minor']
#  ['G Major', 'E Minor']
#  ['A Major', 'F♯ Minor']
#  ['B Major', 'G♯ Minor']
#  ['C♯ Major', 'A♯ Minor']
#  ['F♯ Major', 'D♯ Minor']
#  ['C Minor', 'Eb Major']
#  ['F Minor', 'Ab Major']
#  ['G Minor', 'Bb Major']
#  ['Cb Major', 'Ab Minor']
#  ['Db Major', 'Bb Minor']
#  ['Gb Major', 'Eb Minor']

Key Identification

In [33]:
#A dictionary of how many times each unique note appeared in all octaves
midi_soup_dict = {}
for note, count in note_soup:
    # Extract the note name without octave
    note_name_without_octave = note[:-1]
    midi_soup_dict[str(librosa.note_to_midi(note_name_without_octave))] = midi_soup_dict.get(str(librosa.note_to_midi(note_name_without_octave)),0)+count #this is already sorted
    uniq_significat_midi_sets = [str(keys) for keys in midi_soup_dict.keys()]


In [34]:
#create a dictionary of scores to observe the notes that come closest to the key
key_score_dict = {}
for key in key_scales:
  key_score_dict[key]= 0 # intialize with zero

In [35]:
for key,value_set in key_scales.items():
  value_list = [str(librosa.note_to_midi(note)) for note in value_set] #global keys
  uniq_notes_set = set(uniq_significat_midi_sets) #local keys
  dif_in_keys = uniq_notes_set-set(value_list)
  different_midi_error = sum([midi_soup_dict.get(note_def,0) for note_def in list(dif_in_keys)])
  key_score_dict[key] = different_midi_error/len(dif_in_keys) if not len(dif_in_keys) == 0 else 0
  sorted_key_score_dict = {k: v for k, v in sorted(key_score_dict.items(), key=lambda item: item[1])}

In [36]:
#Handling for keys that receive the same error score
#Get the keys with the lowest error values
keys_with_value = [key for key, value in sorted_key_score_dict.items() if value == sorted_key_score_dict.get(min(sorted_key_score_dict,key=lambda k:sorted_key_score_dict[k]))]

In [37]:
#storing in a list the keys that have the lowest error rate
probable_keys = [key_scales[key] for key in keys_with_value]
probable_midi_keys = [list(librosa.note_to_midi(note_list)) for note_list in probable_keys]
#converting the unique notes of the song to midi numbers to identify the lowest note
unique_midi = librosa.note_to_midi(unique_notes)
#sort from smallest to largest
unique_midi.sort()
#take the midi numbers back to note name
sorted_notes = librosa.midi_to_note(unique_midi)
#remove the octaves from the notes and take it back to midi notes so that we get the midi numbers of only pure notes at zero octave
sort_note = [librosa.note_to_midi(note[:-1]) for note in sorted_notes]
# Find the lowest note of the song that appear in all sublists
common_value = [value for value in sort_note if all(value in sublist for sublist in probable_midi_keys)][0]
#find the position of the lowest common_value that appears first in all the sublist
positions = [(index, sublist.index(common_value)) for index, sublist in enumerate(probable_midi_keys) if common_value in sublist]

#extract the the name of the keys based on the notes # == extract the values of the keys based on the index #nested 1 [#extract the sublist from the probable keys where the lowest common values appears first #nested 2]
detected_musical_key = [key for key, value in key_scales.items() if list(librosa.note_to_midi(value)) == probable_midi_keys[min((index for index, position in positions), key=lambda x: positions[x][1])]]
accepted_notes = key_scales.get(detected_musical_key[0])

#the sets of notes converted to midi notes and sorted
accepted_midi = [str(librosa.note_to_midi(n)) for n in accepted_notes]
accepted_midi.sort()

print(f"The probable keys were {keys_with_value},\nlowest note {librosa.midi_to_note(common_value)[:-1]} which appears first in Detected key: {detected_musical_key[0]}\nwith notes: {accepted_notes}")


The probable keys were ['Db Major', 'Bb Minor'],
lowest note G♯ which appears first in Detected key: Db Major
with notes: ['Db', 'Eb', 'F', 'Gb', 'Ab', 'Bb', 'C']


Note Quantization

In [38]:
#Identify the index and the frequencies of the notes that need to be Quantized

not_key_note = list(set(uniq_significat_midi_sets).difference(accepted_midi))#notes in the song not in the key
note_name = [str(librosa.note_to_midi(note[:-1])) for note in y_notes]
index_to_change = [np.where(np.array(note_name) == not_key_note[i])[0] for i in range(len(not_key_note))]
indices = np.concatenate(index_to_change)
indices_sort = np.sort(indices)
freq_to_change = [freq[index] for index in  indices_sort]

In [39]:
#finds the notes in the music that does not appear in the key and then transforms it to the closest note in the key

def find_closest_midi(input_number, num_list = accepted_midi):
    closest_numbers = []
    min_difference = float('inf')

    for num in num_list:
        num = int(num)
        difference = abs(input_number - num)

        if difference < min_difference:
            min_difference = difference
            closest_numbers = [num]
        elif difference == min_difference:
            closest_numbers.append(num)

    return closest_numbers


#creates a list of the notes that has been transformed using find_closest_midi function
def quantize_note(note:str,freq=0):

  pure_note = note[:-1]
  pure_scale = note[-1:]
  pure_midi = librosa.note_to_midi(pure_note)
  closest_midis = find_closest_midi(pure_midi)
  closest_midi_notes = librosa.midi_to_note(closest_midis)
  result_notes = [c_n[:-1]+pure_scale for c_n in closest_midi_notes]
  result_frequency = [librosa.note_to_hz(note) for note in result_notes]
  if freq:
    diff = abs(result_frequency - freq)
    closest=result_frequency[np.where(diff == min(diff))[0][0]]
    return  librosa.hz_to_midi(closest)

  return librosa.note_to_midi(result_notes[-1])

degrees = []
frequencies = []
for i,(note,fre) in enumerate(zip(y_notes,freq)):
  if i in indices_sort:
    closest_key_note = quantize_note(note,fre)
  else:
    closest_key_note = quantize_note(note)
  degrees.append(closest_key_note)




In [40]:
detected_key = detected_musical_key[0]
key_note = detected_key.split(" ")[0]
key_note_in_soup = None
for (note,occurence) in note_soup:
  if librosa.note_to_midi(note[:-1]) == librosa.note_to_midi(key_note):
    key_note_in_soup = note
    break #gets the first note with octave corresponding to key_note

#the quantized notes are now reset to the correct octave
quantized_degrees = [int(round(deg,1)) for deg in degrees]

Scale quantization


In [56]:
quantized_notes = librosa.midi_to_note(quantized_degrees)

In [85]:
octave_range = [int(min(note[-1:] for note in quantized_notes)),int(max(note[-1:] for note in quantized_notes))]
octave_mean = np.mean([int(note[-1:]) for note in quantized_notes])
octave_mode = st.mode([int(note[-1:]) for note in quantized_notes])
data = [int(note[-1:]) for note in quantized_notes]
print(octave_mode)
confidence_interval = st.t.interval(0.10, len(data)-1, np.mean(data), st.sem(data))# scale=st.sem(data)
confidence_interval

ModeResult(mode=4, count=246)


(3.6749219607815924, 3.685781851535123)

Time Quantization and metronome

In [41]:
#Time Quantization

bpm = tempo
beat_per_sec = bpm/60
second_per_beat = 60/bpm
grids_per_beat = 4
seconds_per_grid = (second_per_beat/4)
msec_per_grid = (second_per_beat/4)*1000

corrected_start_time_grid = [round(x/seconds_per_grid) for x in onset_times]
duplicates = [i for i, x in enumerate(corrected_start_time_grid) if corrected_start_time_grid.count(x) > 1]
index_to_change_new = [duplicates[i] for i in range(len(duplicates)) if i%2!=0]

#takes the index to change and then pushes the value back by one value: (the corrected_start_time_grid is being changed in this step)
for x in index_to_change_new:
  runnig_index = x
  while runnig_index>=0:

    if corrected_start_time_grid[runnig_index] == corrected_start_time_grid[runnig_index-1] and corrected_start_time_grid[runnig_index-1]!=0:
      corrected_start_time_grid[runnig_index-1]=corrected_start_time_grid[runnig_index-1]-1
      runnig_index = runnig_index-1

    else:
      break;

#find the positions(index) where the value of the grid is zero
something = [i for i,x in enumerate(corrected_start_time_grid) if x == 0]

#increasing the values of the index by an increment of 1 from the index of the last value till the index of the first value
for s in something[::-1]:
  if s==0:
    break;
  for i in range(s, len(corrected_start_time_grid)):
    corrected_start_time_grid[i] += 1

#change the grid values to seconds and then beat for midi conversion
corrected_start_time = [x*seconds_per_grid for x in corrected_start_time_grid]
corrected_start_time_beat = [x * beat_per_sec for x in corrected_start_time]
duration_in_second = [onset_times[i]-onset_times[i-1] for i in range(1,len(onset_times))]
corrected_start_time_beat.pop()

#truncate over-lapping notes
for i in range(len(corrected_start_time)-1):
  if corrected_start_time[i] + duration_in_second[i] > corrected_start_time[i+1]:
    duration_in_second[i] =  corrected_start_time[i+1] - corrected_start_time[i]

duration_beat_trunc = [x * beats_per_sec for x in duration_in_second]


In [42]:
second_per_beat = 60/bpm #1 beat = 1 second
grids_per_beat = 4 # 1 beat = 4 grids = 1 second
seconds_per_grid = (second_per_beat/grids_per_beat) #1 grid = 0.25 seconds

grid_times = [round(x/seconds_per_grid) for x in onset_times] #the grid index of where the onset times will align #this is just a count
last_grid_val = max(grid_times)
length = last_grid_val + 4 if last_grid_val % 4!=0 else last_grid_val
metronome = range(0,length)
metronome_grid = [i for i, x in enumerate(metronome) if i % 4 == 0]
metronome_grid_seconds = [x*seconds_per_grid for x in metronome_grid]#converted the positions into seconds
metronome_grid_beat = [x * beat_per_sec for x in metronome_grid_seconds]#converted the seconds into beats

metronome_degrees = [librosa.note_to_midi(most_common_note)]*(len(metronome_grid_beat)-1)
metronome_duration = [metronome_grid_beat[1]/2]*(len(metronome_grid_beat)-1)
metronome_vol = [100] * (len(metronome_grid_beat)-1)


Chord Generator

In [43]:
def generate_chords(scale:str):
  key_notes=key_scales.get(scale)
  length = len(key_notes)
  triad_chords = []
  for i in range(length):
    triad_chords.append([key_notes[i%length],key_notes[(i+2)%length],key_notes[(i+4)%length]])
  return triad_chords

Holding Chords

In [44]:
def sort_by_note_position(triad,note):
    return triad.index(note)

key_chords = generate_chords(detected_key)
midi_chords = []
for c_l in key_chords:
  cell = []
  for n in c_l:
    cell.append(librosa.note_to_midi(n))
  midi_chords.append(cell)

q_notes = librosa.midi_to_note(quantized_degrees)

chord_prog =[]
chord_duration = []
chord_start = []
detected_chords = []
this_chord = []
prev_chord = []
start = 0

#extract octave from the most occuring note
chord_octave = int((most_common_note)[-1])

for i,(note, time) in enumerate(zip(q_notes, corrected_start_time_beat)):
  if i == len(q_notes)-1:
    chord_start.append(corrected_start_time_beat[start])
    chord_duration.append(corrected_start_time_beat[i]-corrected_start_time_beat[start])
    chord_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave) for x in prev_chord])

  if not detected_chords:
      detected_chords = midi_chords

  prev_chord = detected_chords[0]
  detected_chords = sorted([x for x in detected_chords if librosa.note_to_midi(note[:-1]) in x], key = lambda triad: sort_by_note_position(triad, librosa.note_to_midi(note[:-1])))
  this_chord = detected_chords[0] if detected_chords else None

  if not this_chord:
    chord_start.append(corrected_start_time_beat[start])
    chord_duration.append(corrected_start_time_beat[i]-corrected_start_time_beat[start])

    start = i
    chord_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave) if x>12 else librosa.midi_to_note(x+12)[:-1]+str(chord_octave) for x in prev_chord])
    detected_chords = midi_chords
    detected_chords = sorted([x for x in detected_chords if librosa.note_to_midi(note[:-1]) in x], key=lambda triad: sort_by_note_position(triad, librosa.note_to_midi(note[:-1])))




Sustaining chords

In [45]:
midi_chords=[]
for c_l in key_chords:
  cell=[]
  for n in c_l:
    cell.append(librosa.note_to_midi(n))
  midi_chords.append(cell)

chord_spike_prog =[]
chord_spike_duration = []
chord_spike_start = []
detected_chords = []
start = 0

for i,(note, time) in enumerate(zip(q_notes, corrected_start_time_beat)):
  if not detected_chords:
    detected_chords = midi_chords
  detected_chords = sorted([x for x in midi_chords if librosa.note_to_midi(note[:-1]) in x], key=lambda triad: sort_by_note_position(triad, librosa.note_to_midi(note[:-1])))
  chord_spike  = detected_chords[0]
  chord_spike_start.append(corrected_start_time_beat[i])
  if i < len(corrected_start_time_beat)-1:
    chord_spike_duration.append(corrected_start_time_beat[i+1]-corrected_start_time_beat[i])
  else :
    chord_spike_duration.append(.25)
  chord_spike_prog.append([librosa.midi_to_note(x)[:-1]+str(chord_octave) if x>12 else librosa.midi_to_note(x+12)[:-1]+str(chord_octave)  for x in chord_spike])

prev_chord = None
smoothen_chords = []
smoothen_start =[]
smoothen_duration=[]
start_i=0
dur_i=0
for i,(chord, start,dur) in enumerate(zip(chord_spike_prog,chord_spike_start,chord_spike_duration)):
  if not prev_chord:

    prev_chord = chord
    start_i=start
    dur_i=0

  if prev_chord != chord:
    smoothen_chords.append(prev_chord)
    smoothen_start.append(start_i)
    smoothen_duration.append(dur_i)
    start_i=start
    dur_i=dur
    prev_chord = chord

  else:
    dur_i += dur


smoothen_vol = [100] * (len(smoothen_chords))




In [46]:
#Writing the midi file

# Create a MIDIFile object
midi = MIDIFile(numTracks=6)

# Set track and channel for the audio
track = 0
channel = 1
program = 0 #
time = 0
track_name = "Piano"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for pitch, time, duration, volume in zip(quantized_degrees, corrected_start_time_beat, duration_beat_trunc, norm_vol):
  # print(time,duration)
  midi.addNote(track, channel, pitch, time, duration, volume)


# Set track and channel for the Chord(Major-third)(Hold)
track = 2
channel = 3
program = 0# piano
time = 0
track_name = "Major_third_chord_Hold"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(chord_prog, chord_start, chord_duration, norm_vol):
  # print(time,duration)
  for c_n in chord:
    midi.addNote(track, channel, librosa.note_to_midi(c_n), time, duration, volume)

# Set track and channel for the Hold chord bassline
track = 1
channel = 2
program = 32
time = 0
track_name = "Hold_Bass_line"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(chord_prog, chord_start, chord_duration, norm_vol):
  # print(time,duration)
    midi.addNote(track, channel, min(librosa.note_to_midi(chord)), time, duration, volume)



# Set track and channel for the Chord(Major-third)(Sustain)
track = 4
channel = 5
program = 0#piano
time = 0
track_name = "Major_third_chord_smoothen"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo = int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(smoothen_chords, smoothen_start, smoothen_duration, smoothen_vol):
   for c_n in chord:
    midi.addNote(track, channel, librosa.note_to_midi(c_n), time, duration, volume)

# Set track and channel for the Smoothen chord bassline
track = 3
channel = 4
program = 32
time = 0
track_name = "Smoothen_Bass_line"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
# Add the notes to the MIDI file
for chord, time, duration, volume in zip(smoothen_chords, smoothen_start, smoothen_duration, smoothen_vol):
  # print(time,duration)
    midi.addNote(track, channel, min(librosa.note_to_midi(chord)), time, duration, volume)


# Set track and channel for the metronome
track = 5
channel = 6
program = 115
time = 0
track_name = "Metronome"
midi.addTrackName(track, time, track_name)
midi.addProgramChange(track, channel, 0, program)

midi.addTempo(track, 0, tempo= int(tempo))
metronome_grid_beat.pop()
# Add the notes to the MIDI file
for pitch, time, duration, volume in zip(metronome_degrees, metronome_grid_beat, metronome_duration, metronome_vol):
  midi.addNote(track, channel, pitch, time, duration, volume)


# Write the MIDI file to disk

output_file = detected_musical_key[0] +'_'+ str(tempo)+'bpm'+'_'+audio_file.split('/')[2].split(".")[0]+ '.mid'
with open(output_file, 'wb') as file:
    midi.writeFile(file)

In [47]:
librosa.midi_to_note(quantized_degrees)

array(['F4', 'F4', 'C4', 'C♯4', 'D♯4', 'F4', 'D♯4', 'F4', 'F4', 'D♯4',
       'D♯4', 'F4', 'F4', 'D♯4', 'F4', 'D♯4', 'F4', 'F4', 'C♯4', 'A♯3',
       'C♯4', 'C4', 'D♯4', 'D♯4', 'F4', 'D♯4', 'C♯4', 'D♯4', 'A♯3', 'C♯4',
       'D♯4', 'D♯4', 'C♯4', 'A♯4', 'D♯4', 'F4', 'D♯4', 'C♯4', 'D♯4',
       'D♯4', 'A♯3', 'A♯3', 'A♯5', 'A♯2', 'D♯4', 'C♯4', 'D♯4', 'F4', 'F4',
       'G♯4', 'F4', 'D♯4', 'F4', 'D♯4', 'C♯4', 'F4', 'D♯4', 'C♯4', 'A♯3',
       'A♯3', 'A♯3', 'A♯3', 'C♯4', 'D♯4', 'D♯4', 'C♯4', 'D♯4', 'F4',
       'D♯4', 'D♯4', 'D♯4', 'A♯3', 'C♯4', 'C♯4', 'D♯4', 'F4', 'G♯5', 'F4',
       'D♯4', 'D♯4', 'F♯4', 'F4', 'D♯4', 'F4', 'D♯4', 'F4', 'F4', 'C♯4',
       'A♯3', 'C♯4', 'C♯4', 'D♯4', 'F4', 'F4', 'C♯4', 'D♯4', 'D♯4', 'A♯3',
       'C♯4', 'C♯4', 'D♯4', 'D♯4', 'C♯4', 'C♯4', 'C♯5', 'C5', 'C5', 'D♯6',
       'D♯4', 'F4', 'D♯4', 'D♯4', 'C♯4', 'F4', 'F4', 'F4', 'C♯4', 'A♯3',
       'A♯3', 'A♯3', 'A♯5', 'A♯2', 'D♯4', 'D♯4', 'C♯4', 'F4', 'D♯4',
       'G♯4', 'F4', 'D♯3', 'F4', 'D♯4', 'C♯3', 'D♯4', '

In [48]:
for chord, beat in zip(chord_prog, chord_duration):
  print(chord, beat)


['F4', 'G♯4', 'C4'] 2.0
['C♯4', 'F4', 'G♯4'] 2.5
['D♯4', 'F♯4', 'A♯4'] 1.75
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 2.25
['F4', 'G♯4', 'C4'] 1.75
['D♯4', 'F♯4', 'A♯4'] 1.25
['F4', 'G♯4', 'C4'] 4.0
['D♯4', 'F♯4', 'A♯4'] 0.5
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 1.0
['A♯4', 'C♯4', 'F4'] 5.75
['C4', 'D♯4', 'F♯4'] 3.0
['F4', 'G♯4', 'C4'] 0.5
['D♯4', 'F♯4', 'A♯4'] 0.75
['C♯4', 'F4', 'G♯4'] 2.5
['D♯4', 'F♯4', 'A♯4'] 2.25
['C♯4', 'F4', 'G♯4'] 2.5
['D♯4', 'F♯4', 'A♯4'] 2.0
['A♯4', 'C♯4', 'F4'] 4.75
['D♯4', 'F♯4', 'A♯4'] 1.5
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 1.25
['C♯4', 'F4', 'G♯4'] 1.5
['D♯4', 'F♯4', 'A♯4'] 9.0
['C♯4', 'F4', 'G♯4'] 1.25
['D♯4', 'F♯4', 'A♯4'] 0.5
['F4', 'G♯4', 'C4'] 5.0
['D♯4', 'F♯4', 'A♯4'] 1.5
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 1.0
['C♯4', 'F4', 'G♯4'] 3.25
['D♯4', 'F♯4', 'A♯4'] 0.75
['A♯4', 'C♯4', 'F4'] 5.75
['D♯4', 'F♯4', 'A♯4'] 2.75
['C♯4', 'F4', 'G♯4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 0.75
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 2.5
['

In [49]:
for chord, beat in zip(smoothen_chords, smoothen_duration):
  print(chord, beat)

['F4', 'G♯4', 'C4'] 1.25
['C4', 'D♯4', 'F♯4'] 0.75
['C♯4', 'F4', 'G♯4'] 2.5
['D♯4', 'F♯4', 'A♯4'] 1.75
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 2.25
['F4', 'G♯4', 'C4'] 1.75
['D♯4', 'F♯4', 'A♯4'] 1.25
['F4', 'G♯4', 'C4'] 4.0
['D♯4', 'F♯4', 'A♯4'] 0.5
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 1.0
['F4', 'G♯4', 'C4'] 2.0
['C♯4', 'F4', 'G♯4'] 1.25
['A♯4', 'C♯4', 'F4'] 1.0
['C♯4', 'F4', 'G♯4'] 1.5
['C4', 'D♯4', 'F♯4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 2.0
['F4', 'G♯4', 'C4'] 0.5
['D♯4', 'F♯4', 'A♯4'] 0.75
['C♯4', 'F4', 'G♯4'] 2.5
['D♯4', 'F♯4', 'A♯4'] 1.5
['A♯4', 'C♯4', 'F4'] 0.75
['C♯4', 'F4', 'G♯4'] 2.5
['D♯4', 'F♯4', 'A♯4'] 2.0
['C♯4', 'F4', 'G♯4'] 3.5
['A♯4', 'C♯4', 'F4'] 1.25
['D♯4', 'F♯4', 'A♯4'] 1.5
['F4', 'G♯4', 'C4'] 1.0
['D♯4', 'F♯4', 'A♯4'] 1.25
['C♯4', 'F4', 'G♯4'] 1.5
['D♯4', 'F♯4', 'A♯4'] 2.0
['A♯4', 'C♯4', 'F4'] 6.0
['D♯4', 'F♯4', 'A♯4'] 1.0
['C♯4', 'F4', 'G♯4'] 1.25
['D♯4', 'F♯4', 'A♯4'] 0.5
['F4', 'G♯4', 'C4'] 2.0
['G♯4', 'C4', 'D♯4'] 1.75
['F4', 'G♯4', 'C4'] 1.25
['D♯4',